In [132]:
filepath = "e:/#MyStuff#/YogaNidra/"  #Input audio file path
output_filepath = "e:/#MyStuff#/YogaNidra/Transcripts/" #Final transcript path
bucketname = "audiofiles123456" #Name of the bucket created in the step before

# Import libraries
from pydub import AudioSegment
AudioSegment.converter = "E:/ffmpeg/bin"
AudioSegment.ffmpeg = "E:/ffmpeg/bin"
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import wave
from google.cloud import storage

In [133]:
def mp3_to_wav(audio_file_name):
    if audio_file_name.split('.')[-1] == 'mp3': 
        print(audio_file_name + '   It\'s mp3')       
        subprocess.call(['ffmpeg', '-i', audio_file_name.split('.')[0] + '.mp3',
                   audio_file_name.split('.')[0] + '.wav'])

In [134]:
def stereo_to_mono(audio_file_name):
    sound = AudioSegment.from_wav(audio_file_name)
    sound = sound.set_channels(1)
    sound.export(audio_file_name, format="wav")

In [135]:
def frame_rate_channel(audio_file_name):
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

In [136]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

In [137]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [138]:
def google_transcribe(audio_file_name):
    
    file_name = filepath + audio_file_name
    mp3_to_wav(file_name)
    print('   mp3_to_wav end')
    wav_file_name = file_name.split('.')[0] + '.wav'

    # The name of the audio file to transcribe
    frame_rate, channels = frame_rate_channel(wav_file_name)
    
    if channels > 1:
       stereo_to_mono(wav_file_name)
    
    bucket_name = bucketname
    #source_file_name = filepath + wav_file_name
    destination_blob_name = wav_file_name
    
    upload_blob(bucket_name, wav_file_name, destination_blob_name)
    print('   wav is uploaded')
    
    gcs_uri = 'gs://' + bucketname + '/' + wav_file_name
    transcript = ''
    
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)

    config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='hu-HU')

    # Detects speech in the audio file
    print('   speech-to-text generation is started')
    operation = client.long_running_recognize(config, audio)
    response = operation.result(timeout=10000)
    print('   speech-to-text generation has finished')

    for result in response.results:
        transcript += result.alternatives[0].transcript
    
    delete_blob(bucket_name, destination_blob_name)
    print('   temporary files are deleted')
    return transcript

In [142]:
def write_transcripts(transcript_filename,transcript):
    f= open(output_filepath + transcript_filename,"w+",encoding="utf-8")
    f.write(transcript)
    f.close()
    print('   text printed into file')

In [149]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "E:\#MyProjects#\API-key\speechtotext-278012-1df3445de29b.json"

#to_be_processed = os.listdir(filepath)
to_be_processed = ['200204-yoga-nidra-7-alkalom-1-gyakorlat.mp3',
 '200204-yoga-nidra-7-alkalom-2-gyakorlat.mp3',
 '200303-yoga-nidra-8-alkalom-1_gyakorlat.mp3',
 '200303-yoga-nidra-8-alkalom-2_gyakorlat.mp3',
 '200407-yoga-nidra-9-alkalom-1_gyakorlat.mp3',
 '200415-yoga-nidra-10-alkalom-1-gyakorlat.mp3',
 '200415-yoga-nidra-10-alkalom-2-gyakorlat.mp3'
 ]

if __name__ == "__main__":
    
    for audio_file_name in to_be_processed:
        #audio_file_name = '200303-yoga-nidra-8-alkalom-1_gyakorlat.mp3'
        print(audio_file_name)
        
        transcript = google_transcribe(audio_file_name)
        transcript_filename = audio_file_name.split('.')[0] + '.txt'
        write_transcripts(transcript_filename,transcript)

200204-yoga-nidra-7-alkalom-1-gyakorlat.mp3
e:/#MyStuff#/YogaNidra/200204-yoga-nidra-7-alkalom-1-gyakorlat.mp3   It's mp3
   mp3_to_wav end
   wav is uploaded
   speech-to-text generation is started
   speech-to-text generation has finished
   temporary files are deleted
   text printed into file
200204-yoga-nidra-7-alkalom-2-gyakorlat.mp3
e:/#MyStuff#/YogaNidra/200204-yoga-nidra-7-alkalom-2-gyakorlat.mp3   It's mp3
   mp3_to_wav end
   wav is uploaded
   speech-to-text generation is started
   speech-to-text generation has finished
   temporary files are deleted
   text printed into file
200303-yoga-nidra-8-alkalom-1_gyakorlat.mp3
e:/#MyStuff#/YogaNidra/200303-yoga-nidra-8-alkalom-1_gyakorlat.mp3   It's mp3
   mp3_to_wav end
   wav is uploaded
   speech-to-text generation is started
   speech-to-text generation has finished
   temporary files are deleted
   text printed into file
200303-yoga-nidra-8-alkalom-2_gyakorlat.mp3
e:/#MyStuff#/YogaNidra/200303-yoga-nidra-8-alkalom-2_gyakorlat

In [144]:
transcript

'A talpáról még nem beszéltem Illetve nagyon sok mindenről Lebeszéltem tehát általában egy elhatározást szoktunk a gyakorlás előtt csinálni ezt a jógában szanka apának hívjuk most a szanka apánk a következő lesz most én csak elmondom jóga nidrá itt gyakorol Ok éber tudatos mozdulatlan maradok ha van olyan akinek van saját szaggal pálya ki lehet vele egészíteni tehát most Keresd meg a helyedet És várjál mozdulatlanná magadba háromszor Ismételd el a San Kárpát jóga nitrát gyakorol Ok éber tudatos mozdulatlan maradok pizza figyelmedet a homlokodra lazítsd el hozd ki belőle a feszültséget simítsd ki a homlok adat siminszky a homlok ráncokat és az elme ráncait lazítsd a jobb szemöldököt és lazítsd apa lazítsd a jobb felső alsó szemhéjat lazítsd a bal felső alsó szemhéj lazítsd A jobb szemgolyót és a számokat lazítsd a bal szemgolyót és a szemizmok lazítsd az óra lazítsd a jobb orcád Fradista pap lazítsd a felső ajkat bázist az ásót lazítsd a jobb szájukat fazist apa most a tudatosságot de l

In [128]:
transcript_filename = audio_file_name.split('.')[0] + '.txt'
write_transcripts(transcript_filename,transcript)

200303-yoga-nidra-8-alkalom-1_gyakorlat.txt


In [147]:
os.listdir(filepath)

['190507-yoga-nidra-1-gyakorlat.mp3',
 '190507-yoga-nidra-1-gyakorlat.wav',
 '190507-yoga-nidra-2-gyakorlat noise reduced.mp3',
 '190507-yoga-nidra-2-gyakorlat noise reduced.wav',
 '190507-yoga-nidra-2-gyakorlat.mp3',
 '190507-yoga-nidra-2-gyakorlat.wav',
 '190604-yoga-nidra-2-alkalom-1-gyakorlat.mp3',
 '190604-yoga-nidra-2-alkalom-1-gyakorlat.wav',
 '190604-yoga-nidra-2-alkalom-2-gyakorlat.mp3',
 '190604-yoga-nidra-2-alkalom-2-gyakorlat.wav',
 '190702-yoga-nidra-3-alkalom-1-gyakorlat.mp3',
 '190702-yoga-nidra-3-alkalom-1-gyakorlat.wav',
 '190702-yoga-nidra-3-alkalom-2-gyakorlat.mp3',
 '190702-yoga-nidra-3-alkalom-2-gyakorlat.wav',
 '190903-yoga-nidra-4-alkalom-1-gyakorlat.mp3',
 '190903-yoga-nidra-4-alkalom-1-gyakorlat.wav',
 '190903-yoga-nidra-4-alkalom-2-gyakorlat.mp3',
 '190903-yoga-nidra-4-alkalom-2-gyakorlat.wav',
 '191001-yoga-nidra-5-alkalom-1-gyakorlat.mp3',
 '191001-yoga-nidra-5-alkalom-1-gyakorlat.wav',
 '200107-yoga-nidra-6-alkalom-1-gyakorlat.mp3',
 '200107-yoga-nidra-6-al